# Install dependancies





Atari dependancies - for pacman testing

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools > /dev/null 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

Rendering dependencies

In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Install Dependencies and Stable Baselines Using Pip

List of full dependencies can be found in the [README](https://github.com/hill-a/stable-baselines).


In [0]:
!apt install swig cmake libopenmpi-dev zlib1g-dev > /dev/null 2>&1
!pip install stable-baselines==2.4.0 box2d box2d-kengz > /dev/null 2>&1

# Imports and Helper functions


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML
import os 

from IPython import display as ipythondisplay

In [0]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

W0614 12:33:57.974799 139753948342144 abstractdisplay.py:144] xdpyinfo was not found, X start can not be checked! Please install xdpyinfo!


<Display cmd_param=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] cmd=['Xvfb', '-br', '-nolisten', 'tcp', '-screen', '0', '1400x900x24', ':1001'] oserror=None return_code=None stdout="None" stderr="None" timeout_happened=False>

In [0]:
from stable_baselines.common.policies import CnnPolicy, CnnLstmPolicy, CnnLnLstmPolicy, MlpPolicy
# from stable_baselines.ddpg.policies import MlpPolicy
from stable_baselines.common.vec_env import DummyVecEnv
from stable_baselines.ppo2 import PPO2
# from stable_baselines.ddpg import DDPG
# from stable_baselines.deepq import DQN, MlpPolicy
# from stable_baselines.ddpg.noise import AdaptiveParamNoiseSpec

# from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy

In [0]:
"""IMPORT PACKAGES"""

from signal import signal, SIGPIPE, SIG_DFL
signal(SIGPIPE, SIG_DFL) 

import argparse

'''Model selection '''
from stable_baselines import logger

# from stable_baselines.ppo2 import PPO2
# from stable_baselines.common.policies import CnnPolicy, CnnLstmPolicy, CnnLnLstmPolicy, MlpPolicy

'''Vectorized Env'''
from stable_baselines.common.vec_env import SubprocVecEnv, DummyVecEnv, VecFrameStack
from stable_baselines.common.cmd_util import make_atari_env
from stable_baselines.common import set_global_seeds

'''Monitoring Learning process'''
# from stable_baselines.bench import Monitor
from stable_baselines.results_plotter import load_results, ts2xy
from stable_baselines.common.atari_wrappers import MaxAndSkipEnv, FrameStack


In [0]:
import itertools as it
from skimage import color

# Functions


define plotting function

In [0]:
'''Plotting'''

def movingAverage(values, window):
    """
    Smooth values by doing a moving average
    :param values: (numpy array)
    :param window: (int)
    :return: (numpy array)
    """
    weights = np.repeat(1.0, window) / window
    return np.convolve(values, weights, 'valid')


def plot_results(log_folder, title='Learning Curve', smoothing = True):
    """
    plot the results

    :param log_folder: (str) the save location of the results to plot
    :param title: (str) the title of the task to plot
    """
    x, y = ts2xy(load_results(log_folder), 'timesteps')

    if smoothing:
        y = movingAverage(y, window=50)
    else:
        title = 'Learning Curve no smoothing'

    # Truncate x
    x = x[len(x) - len(y):]

    fig = plt.figure(title)
    plt.plot(x, y)
    plt.xlabel('Number of Timesteps')
    plt.ylabel('Rewards')
    plt.title(title + " Smoothed")
    plt.show()

define evaluation function

In [0]:
def evaluate(model, env, num_steps=1000):
    """
    Evaluate a RL agent
    :param model: (BaseRLModel object) the RL Agent
    :param num_steps: (int) number of timesteps to evaluate it
    :return: (float) Mean reward for the last 100 episodes
    """
    episode_rewards = [0.0]
    obs = env.reset()
    for i in range(num_steps):
        # _states are only useful when using LSTM policies
        action, _states = model.predict(obs)
        # here, action, rewards and dones are arrays
        # because we are using vectorized env
        obs, rewards, dones, info = env.step(action)

        # Stats
        episode_rewards[-1] += rewards[0]
        if dones[0]:
            obs = env.reset()
            episode_rewards.append(0.0)
    # Compute mean reward for the last 100 episodes
    mean_100ep_reward = round(np.mean(episode_rewards[-100:]), 1)
    print("Mean reward:", mean_100ep_reward, "Num episodes:", len(episode_rewards))

    return mean_100ep_reward

define callback function

In [0]:
def callback(_locals, _globals):
  """
  Callback called at each step (for DQN an others) or after n steps (see ACER or PPO2)
  :param _locals: (dict)
  :param _globals: (dict)
  """
  global n_steps, best_mean_reward
  # Print stats every 1000 calls
  if (n_steps + 1) % 100000 == 0:
      # Evaluate policy performance
      x, y = ts2xy(load_results(log_dir), 'timesteps')
      if len(x) > 0:
          mean_reward = np.mean(y[-100:])
          print(x[-1], 'timesteps')
          print("Best mean reward: {:.2f} - Last mean reward per episode: {:.2f}".format(best_mean_reward, mean_reward))

          # New best model, you could save the agent here
          if mean_reward > best_mean_reward:
              best_mean_reward = mean_reward
              # Example for saving best model
              print("Saving new best model")
              _locals['self'].save(log_dir + 'best_model.pkl')
  n_steps += 1
  return True

define video function

In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
#   env = Monitor(env, './video', allow_early_resets=True)
#   env = Monitor(env, './video')
  return env

# DQN agent

# Tweak Environment
from source 'CarRacing - master'
link will be provided

In [0]:
all_actions = np.array([k for k in it.product([-1, 0, 1], [1, 0], [0.2, 0])])
print(all_actions)

[[-1.   1.   0.2]
 [-1.   1.   0. ]
 [-1.   0.   0.2]
 [-1.   0.   0. ]
 [ 0.   1.   0.2]
 [ 0.   1.   0. ]
 [ 0.   0.   0.2]
 [ 0.   0.   0. ]
 [ 1.   1.   0.2]
 [ 1.   1.   0. ]
 [ 1.   0.   0.2]
 [ 1.   0.   0. ]]


In [0]:
class CarRacing(PPO2):
    """
    CarRacing specifig part of the DQN-agent

    Some minor env-specifig tweaks but overall
    assumes very little knowledge from the environment
    """

    def __init__(self, max_negative_rewards=100):
        all_actions = np.array(
            [k for k in it.product([-1, 0, 1], [1, 0], [0.2, 0])]
        )
        # car racing env gives wrong pictures without render
        kwargs["render"] = True
        super().__init__(
            action_map=all_actions,
            pic_size=(96, 96)
        )

        self.gas_actions = np.array([a[1] == 1 and a[2] == 0 for a in all_actions])
        self.break_actions = np.array([a[2] == 1 for a in all_actions])
        self.n_gas_actions = self.gas_actions.sum()
        self.neg_reward_counter = 0
        self.max_neg_rewards = max_negative_rewards

    @staticmethod
    def process_image(obs):
        return 2 * color.rgb2gray(obs) - 1.0

    def get_random_action(self):
        """
        Here random actions prefer gas to break
        otherwise the car can never go anywhere.
        """
        action_weights = 14.0 * self.gas_actions + 1.0
        action_weights /= np.sum(action_weights)

        return np.random.choice(self.dim_actions, p=action_weights)

    def check_early_stop(self, reward, totalreward):
        if reward < 0:
            self.neg_reward_counter += 1
            done = (self.neg_reward_counter > self.max_neg_rewards)

            if done and totalreward <= 500:
                punishment = -20.0
            else:
                punishment = 0.0
            if done:
                self.neg_reward_counter = 0

            return done, punishment
        else:
            self.neg_reward_counter = 0
            return False, 0.0

# Main Function

Building Environment

In [0]:
def main():
    """
    Train and save the DQN model, for the cartpole problem

    :param args: (ArgumentParser) the input arguments
    """
    print("Making a new model")


    env = wrap_env(gym.make('CarRacing-v0'))
    env = MaxAndSkipEnv(env, skip=4)

    ''' For framestack(lstm)   '''
    # env = FrameStack(env, 4)
    # env = Monitor(env, log_dir, allow_early_resets=True)
    env = DummyVecEnv([lambda: env])
#     env = VecFrameStack(make_atari_env(env_id=env, num_env=, seed=777), 4)
    model = CarRacing(policy=CnnPolicy, env=env, n_steps=512, nminibatches=256,
                 lam=0.95, gamma=0.99, noptepochs=8, ent_coef=0.01,
                 learning_rate=5e-4, cliprange= lambda f : f * 0.2, verbose=0)

    # mean_reward_before_train = evaluate(model, env=env, num_steps=10000)
    print("Learning started. It takes some time...")
    model.learn(total_timesteps=1000, callback=callback)
    # mean_reward = evaluate(model, env=env, num_steps=10000)
    print("Saving model to CarRacing_model.pkl")
    model.save("CarRacing_model_PPO2_5")
    print("Plotting Learning Curve")
    plot_results(log_dir)
    plot_results(log_dir, smoothing=False)

In [0]:
def run():
    """
    Run a trained model for the pong problem
    """
    env = wrap_env(gym.make('CarRacing-v0'))
    print("wrapping complete")
    env = DummyVecEnv([lambda: env])

#     model = PPO2.load("CarRacing_model_PPO1_"+ str(5) +".pkl", env)
#     avg_rew = evaluate(model=model, env=env, num_steps=10000)
#     model = PPO2.load("CarRacing_model_PPO2_5.pkl", env)
    model = CarRacing.load("CarRacing_model_PPO2_5.pkl", env)
    while True:
        obs, done = env.reset(), False
        episode_rew = 0
        while not done:
            env.render()
            action, _ = model.predict(obs)
            obs, rew, done, _ = env.step(action)

            episode_rew += rew
        if done: 
          break;
        print("Episode reward", episode_rew)
    show_video()

# Run

In [0]:
if __name__ == '__main__':
    # Create log dir
    log_dir = "data/"
    os.makedirs(log_dir, exist_ok=True)
    best_mean_reward, n_steps = -np.inf, 0

#     run()
    main()


Making a new model


TypeError: ignored

# Others

Pacman - for test

In [0]:
# env = wrap_env(gym.make("MsPacman-v0"))
# #check out the pacman action space!
# print(env.action_space)
# observation = env.reset()

# while True:
  
#     env.render()
    
#     #your agent goes here
#     action = env.action_space.sample() 
         
#     observation, reward, done, info = env.step(action) 
   
        
#     if done: 
#       break;
            
# env.close()
# show_video()